In [2]:
####### Import library and models 
import numpy as np;
import pandas as pd;
import xlrd;
import openpyxl;

# Execute command 'pip install info_gain' and 'pip install matplotlib'
!pip install info-gain
!pip install matplotlib

####### Part 1. Data Preprocessing

##### 1.1 Change the original 4 '.csv' data files into '.xlsx' format.

##### 1.2 Import Data

### Read the 4 original data files into pandas data frame.
deals = pd.read_excel("deals.xlsx")
clients = pd.read_excel("clients.xlsx")
assets = pd.read_excel("assets.xlsx")

# Change the field name 'assetid' to 'AssetID'. Otherwise, it will have problem in grouping statistics.
asset_transactions = pd.read_excel("asset_transaction_history.xlsx")

### Print the record count and head names of each file.
print ("The record count of 'deals' table is: ")
print (deals.shape[0])

print ("The record count of 'clients' table is: ")
print (clients.shape[0])

print ("The record count of 'assets' table is: ")
print (assets.shape[0])

print ("The record count of 'asset_transactions' table is: ")
print (asset_transactions.shape[0])


##### 1.3 Priliminary filters asset transaction frame to get IA asset transactions.

### Remove the deal without any effective date (AccountingDate, BuyerSelectionDate, InitialBidDueDate, MarketingDate, HireDate).

# Get valid deals.
valid_deals = deals [(deals.AccountingDate.notnull()) | (deals.BuyerSelectionDate.notnull()) | (deals.InitialBidDueDate.notnull()) | (deals.MarketingDate.notnull()) | (deals.HireDate.notnull())].drop_duplicates()
print ("The valid record count of 'deals' table is: ")
print (valid_deals.shape[0])

# Get valid IA asset transactions.
ia_asset_transactions = asset_transactions [asset_transactions.DealID.isin (valid_deals.DealID)].drop_duplicates()
print ("The valid record count of 'ia_asset_transactions' table is: ")
print (ia_asset_transactions.shape[0])

# Get valid customer count for each IA asset transaction.
grouped_ia_transactions = ia_asset_transactions.groupby(['DealID', 'AssetID']).size().reset_index(name='ClientCount').drop_duplicates()
print ("The valid record count of 'grouped_asset_transactions' table is: ")
print (grouped_ia_transactions.shape[0])

# Get client data only for IA transactions.
ia_clients = clients [clients.ClientID.isin (ia_asset_transactions.ClientID)].drop_duplicates()
print ("The record count of 'ia_clients' table is: ")
print (ia_clients.shape[0])

# Get the unique asset data for IA transactions.
ia_assets = assets [assets.AssetID.isin (grouped_ia_transactions.AssetID)].drop_duplicates()
print ("The record count of 'ia_assets' table is: ")
print (ia_assets.shape[0])

# Get the unique asset data for non-IA transactions.
not_ia_assets = assets [~assets.AssetID.isin (grouped_ia_transactions.AssetID)].drop_duplicates()
print ("The record count of 'not_ia_assets' table is: ")
print (not_ia_assets.shape[0])

# Save the priliminarily processed data to the below .xlsx files.
valid_deals.to_excel('ia_deals.xlsx')
ia_asset_transactions.to_excel('ia_asset_transactions.xlsx')
grouped_ia_transactions.to_excel('ia_asset_transactions_group.xlsx')
ia_clients.to_excel('ia_clients.xlsx')
ia_assets.to_excel('ia_assets.xlsx')
not_ia_assets.to_excel('ia_not_assets.xlsx')


In [ ]:
## Data quality issue exists. For example, For assetID : 202484, the build date is 2018, but the market data is 2017 year, so the 'time to market' is negative values.
## Delete all similar 'Date to market'record values (<= 180)in dataset
## Keep those abnormal value for analysis

### Data quality problem: AssetID 165000

import datetime as dt;

##### 1.4 Genenrate 'Date_to_Market' and 'Date_to_Predict'data for all assets with market transaction records.

### Assume the 'Built' and 'Renovation' Month_Data to be (01-01) instead of (06-30) to reduce data quality issue.

# Calculate the property 'Built' Year, Date and Time.
def date_build(row):
    return dt.datetime(int(row['Year_Built']), 1, 1)

# Calculate the property 'Renovation' Year, Date and Time.
def date_renovation (row):
    if row['Year_Renovated'] > 0:
       return dt.datetime(row['Year_Renovated'], 1, 1)
    else:
       return np.NAN

# Calculate the interval no. of date from property 'Built' till now.
def date_from_built(row):
    return (row['Current_Date'] - row['Date_Built']).days

# Calculate the interval no. of date from property 'Renovation' till now.
def date_from_renovation(row):
    if row['Year_Renovated'] == -1:
        return -1
    else:
        return (row['Prediction_Date'] - row['Date_Renovated']).days
    

### Merge deals and assets to form a large 'Asset - Deal - Date_to_Market' data frame.

# Merge 'valid_deals' with 'grouped_ia_asset_transactions'.
merge_deals_assets_0 = pd.merge(valid_deals, grouped_ia_transactions, on='DealID', how = 'inner').drop_duplicates()

# Continue to merge the above result data frame with 'ia_assets'.
merge_deals_assets_1 = pd.merge(merge_deals_assets_0, ia_assets, on='AssetID',  how = 'left').drop_duplicates()

merge_deals_assets_1.rename(columns={'PropertyType_y': 'PropertyType'}, inplace=True)

# Delete the columns that result in the data quality problem: 'MSACode', 'Class', and 'Amount'          
deal_asset_time = merge_deals_assets_1 [['AssetID', 'ClientCount', 'City', 'State', 'CountryID', 'CityID', 'State.1', 'Country',
                                         'Zip', 'PropertyTypeID', 'PropertyType', 'PropertyTypeGroup', 'YearBuilt', 'AssetLatitude',
                                         'AssetLongitude', 'LandAreaInAcres', 'NumberofFloors', 'NumberofBuildings', 'YearRenovated',
                                         'Size_Acres', 'Size_InLineSqft', 'IsPortfolio', 'PortfolioCount', 'Lat_city', 'Long_city',
                                         'IsGateway', 'Region', 'Size_Sqft', 'Size_Units', 'PropertySize', 'UnitType', 'ClassDateStamp',
                                         'Market_name', 'CityID.1', 'DealID', 'AccountingDate', 'BuyerSelectionDate', 'InitialBidDueDate',
                                         'MarketingDate', 'HireDate']].drop_duplicates()

# Column 'YearBuilt' is very important for modeling and analaysis.
# Drop the rows with 'YearBuilt' value (null or <1700 or >2019) in the merged data frame
# Drop the rows with 'YearRenovated' value (<1700 or > 2019)
deal_asset_time = deal_asset_time [(deal_asset_time.YearBuilt.notnull()) & (deal_asset_time.YearBuilt < 2019) & (deal_asset_time.YearBuilt >= 1700) & (deal_asset_time.YearRenovated.isnull() | ((deal_asset_time.YearRenovated < 2019) & (deal_asset_time.YearRenovated >= 1700)))]

# Calculate 'Deal_Start_Date' to the the least of five data value (AccountingDate, BuyerSelectionDate, InitialBidDueDate, MarketingDate, HireDate).
deal_asset_time['Deal_Start_Date'] = pd.to_datetime(deal_asset_time[['AccountingDate', 'BuyerSelectionDate', 'InitialBidDueDate',
                                         'MarketingDate', 'HireDate']].min(axis=1))

# Calculate 'Deal_Start_Date' to the the largest of five data value (AccountingDate, BuyerSelectionDate, InitialBidDueDate, MarketingDate, HireDate)
deal_asset_time['Deal_End_Date'] = pd.to_datetime(deal_asset_time[['AccountingDate', 'BuyerSelectionDate', 'InitialBidDueDate',
                                         'MarketingDate', 'HireDate']].max(axis=1))

# Calculate the 'Current_Date' and 'Prediction_Date' (1 year after the current date). 
deal_asset_time['Current_Date'] = pd.Timestamp("today")
deal_asset_time['Prediction_Date'] = pd.Timestamp("today") + pd.DateOffset(years=1)

# Calculate the value for columns 'Year_Built', 'Year_Renovated', 'Date_Built', 'Date_Renovated','Date_from_Built' and 'Date_from_Renovation'.
deal_asset_time['Year_Built'] = pd.to_numeric(deal_asset_time['YearBuilt'], downcast='signed').astype(np.int)
deal_asset_time['Year_Renovated'] = pd.to_numeric(np.where(deal_asset_time ['YearRenovated'].isnull(), -1, deal_asset_time['YearRenovated']), downcast='signed').astype(np.int)
deal_asset_time['Date_Built'] = deal_asset_time.apply (lambda row: date_build(row),axis=1)
deal_asset_time['Date_Renovated'] = deal_asset_time.apply (lambda row: date_renovation(row),axis=1)
deal_asset_time['Date_from_Built'] = deal_asset_time.apply (lambda row: date_from_built(row),axis=1)
deal_asset_time['Date_from_Renovation'] = deal_asset_time.apply (lambda row: date_from_renovation(row),axis=1)

# Order the result data frame by 'AssetID','Deal_Start_Date' and 'Deal_End_Date' ascending orders.
deal_asset_time_order = deal_asset_time.sort_values(by = ['AssetID','Deal_Start_Date','Deal_End_Date'])

# Calculate the row no. for 'deal_asset_time_order' data frame
total_rows = deal_asset_time_order.shape[0]

# Add two columns to calculate the interval start date, and interval deal end data between 2 adjacent rows.
# Set up the initial value to be a very big value.
deal_asset_time_order['Interval_Deal_Start_Date'] = 100000
deal_asset_time_order['Interval_Deal_End_Date'] = 100000

# Use the loop to calculate the interval of 'Deal_Start_Date' and 'Deal_End_Date' between 2 neighbor records 
# The value of column 'Interval_Deal_Start_Date' and 'Interval_Deal_End_Date' will be used to judge if data quality issue exists (Mainly refers to the duplicate transaction records.)
for row in range(1, total_rows):
    if deal_asset_time_order['AssetID'].iloc[row] == deal_asset_time_order['AssetID'].iloc[row-1]:
        deal_asset_time_order['Interval_Deal_Start_Date'].iloc[row] = abs((pd.to_datetime(deal_asset_time_order['Deal_Start_Date'].iloc[row]) - pd.to_datetime(deal_asset_time_order['Deal_Start_Date'].iloc[row-1])).days)
        deal_asset_time_order['Interval_Deal_End_Date'].iloc[row] = abs((pd.to_datetime(deal_asset_time_order['Deal_End_Date'].iloc[row]) - pd.to_datetime(deal_asset_time_order['Deal_End_Date'].iloc[row-1])).days)

        if deal_asset_time_order['Interval_Deal_Start_Date'].iloc[row] <= 10:
            deal_asset_time_order['Interval_Deal_Start_Date'].iloc[row] = np.NAN

        if deal_asset_time_order['Interval_Deal_End_Date'].iloc[row] <= 10:
            deal_asset_time_order['Interval_Deal_End_Date'].iloc[row] = np.NAN


# Filter the rows with data quality issues (Duplicate records) by the values of 'Deal_Start_Date' and 'Deal_End_Date'
deal_asset_time_order_filter = deal_asset_time_order[deal_asset_time_order.Interval_Deal_Start_Date.notnull() & deal_asset_time_order.Interval_Deal_End_Date.notnull()].sort_values(by = ['AssetID','Deal_Start_Date','Deal_End_Date']).drop_duplicates()

# Add 2 columns: 'Date_to_Market' and 'Date_to_Predict'.
deal_asset_time_order_filter['Date_to_Market'] = np.NAN
deal_asset_time_order_filter['Date_to_Predict'] = np.NAN

# Calculate the row no. for 'deal_asset_time_order_filter' data frame
new_total_rows = deal_asset_time_order_filter.shape[0]

# Calculate the value of 'Date_to_Market' and 'Date_to_Predict' in the first row.
deal_asset_time_order_filter['Date_to_Market'].iloc[0] = (pd.to_datetime(deal_asset_time_order_filter['Deal_Start_Date'].iloc[0]) - pd.to_datetime(deal_asset_time_order_filter['Date_Built'].iloc[0])).days
if deal_asset_time_order_filter['Interval_Deal_Start_Date'].iloc[1] == 100000:
    deal_asset_time_order_filter['Date_to_Predict'].iloc[0] = (pd.to_datetime(deal_asset_time_order_filter['Prediction_Date'].iloc[0]) - pd.to_datetime(deal_asset_time_order_filter['Deal_End_Date'].iloc[0])).days

# Use loop to calculate the value of 'Date_to_Market' and 'Date_to_Predict' in all other rows.
for row in range(1, new_total_rows):
    if deal_asset_time_order_filter['Interval_Deal_Start_Date'].iloc[row] == 100000 and deal_asset_time_order_filter['Interval_Deal_End_Date'].iloc[row] == 100000:

        deal_asset_time_order_filter['Date_to_Market'].iloc[row] = (pd.to_datetime(deal_asset_time_order_filter['Deal_Start_Date'].iloc[row]) - pd.to_datetime(deal_asset_time_order_filter['Date_Built'].iloc[row])).days
        if row != (new_total_rows - 1):
            if deal_asset_time_order_filter['Interval_Deal_Start_Date'].iloc[row + 1] == 100000:
                deal_asset_time_order_filter['Date_to_Predict'].iloc[row] = (pd.to_datetime(deal_asset_time_order_filter['Prediction_Date'].iloc[row]) - pd.to_datetime(deal_asset_time_order_filter['Deal_End_Date'].iloc[row])).days
        else:
            deal_asset_time_order_filter['Date_to_Predict'].iloc[row] = (pd.to_datetime(deal_asset_time_order_filter['Prediction_Date'].iloc[row]) - pd.to_datetime(deal_asset_time_order_filter['Deal_End_Date'].iloc[row])).days
    else:
        deal_asset_time_order_filter['Date_to_Market'].iloc[row] = (pd.to_datetime(deal_asset_time_order_filter['Deal_Start_Date'].iloc[row]) - pd.to_datetime(deal_asset_time_order_filter['Deal_End_Date'].iloc[row-1])).days

        if row != (new_total_rows - 1):
            if deal_asset_time_order_filter['Interval_Deal_Start_Date'].iloc[row + 1] == 100000:
                deal_asset_time_order_filter['Date_to_Predict'].iloc[row] = (pd.to_datetime(deal_asset_time_order_filter['Prediction_Date'].iloc[row]) - pd.to_datetime(deal_asset_time_order_filter['Deal_End_Date'].iloc[row])).days
        else:
            deal_asset_time_order_filter['Date_to_Predict'].iloc[row] = (pd.to_datetime(deal_asset_time_order_filter['Prediction_Date'].iloc[row]) - pd.to_datetime(deal_asset_time_order_filter['Deal_End_Date'].iloc[row])).days

# Add the column "Is_Transaction_Exist", and set the value of each row to be 1 (Assets with market transaction).
deal_asset_time_order_filter["Is_Transaction_Exist"] = 1

# Save this field for further data visualization using 'R'
deal_asset_time_order_filter.to_csv('ia_deal_assets_time_complete.csv')

print (deal_asset_time_order_filter.shape)
print (list(deal_asset_time_order_filter.head(0)))


In [ ]:
##### 1.5 Genenrate 'Date_to_Predict' for all assets without market transaction records.

### Assume the 'Built' and 'Renovation' Month_Data to be (01-01) instead of (06-30) to reduce data quality issue.

## Drop column 'Class' and 'MSACode' in 'Asset' Table because they cause some data quality issue.
## Drop 'Amount' in 'Deals' table due to data quality issue.

# Calculate the interval date from property 'Built' till the 'Prediction' date (12 months from now on)
def date_to_predict(row):
    return (pd.to_datetime(row['Prediction_Date']) - pd.to_datetime(row['Date_Built'])).days

# Get the asset records without IA Transaction history.
ia_not_assets = pd.read_excel("ia_not_assets.xlsx")[['AssetID', 'City', 'State', 'CountryID', 'CityID', 'State.1', 'Country',
                                         'Zip', 'PropertyTypeID', 'PropertyType', 'PropertyTypeGroup', 'YearBuilt', 'AssetLatitude',
                                         'AssetLongitude', 'LandAreaInAcres', 'NumberofFloors', 'NumberofBuildings', 'YearRenovated',
                                         'Size_Acres', 'Size_InLineSqft', 'IsPortfolio', 'PortfolioCount', 'Lat_city', 'Long_city',
                                         'IsGateway', 'Region', 'Size_Sqft', 'Size_Units', 'PropertySize', 'UnitType', 'ClassDateStamp',
                                         'Market_name', 'CityID.1']].drop_duplicates()

# Column 'YearBuilt' is very important for modeling and analaysis.
# Drop the rows with 'YearBuilt' value (null or <1700 or > 2019) 
# Drop the rows with 'YearRenovated' value (<1700 or > 2019)
ia_not_assets = ia_not_assets [(ia_not_assets.YearBuilt.notnull()) & (ia_not_assets.YearBuilt < 2019) & (ia_not_assets.YearBuilt >= 1700)   & (ia_not_assets.YearRenovated.isnull() | ((ia_not_assets.YearRenovated < 2019) & (ia_not_assets.YearRenovated >= 1700)))]

# Add columns with all 'NULL' value
ia_not_assets["ClientCount"] = np.NAN
ia_not_assets['DealID'] = np.NAN
ia_not_assets['AccountingDate'] = np.NAN
ia_not_assets['BuyerSelectionDate'] = np.NAN
ia_not_assets['InitialBidDueDate'] = np.NAN
ia_not_assets['MarketingDate'] = np.NAN
ia_not_assets['HireDate'] = np.NAN
ia_not_assets['Deal_Start_Date'] = np.NAN
ia_not_assets['Deal_End_Date'] = np.NAN

# Calculate the value for columns 'Current_Date', 'Prediction_Date', 'Year_Built', 'Year_Renovated', 'Date_Built', 'Date_Renovated','Date_from_Built' and 'Date_from_Renovation'.
ia_not_assets['Current_Date'] = pd.Timestamp("today")
ia_not_assets['Prediction_Date'] = pd.Timestamp("today") + pd.DateOffset(years=1)
ia_not_assets['Year_Built'] = pd.to_numeric(ia_not_assets['YearBuilt'], downcast='signed').astype(np.int)
ia_not_assets['Year_Renovated'] = pd.to_numeric(np.where(ia_not_assets ['YearRenovated'].isnull(), -1, ia_not_assets['YearRenovated']), downcast='signed').astype(np.int)
ia_not_assets['Date_Built'] = ia_not_assets.apply (lambda row: date_build(row),axis=1)
ia_not_assets['Date_Renovated'] = ia_not_assets.apply (lambda row: date_renovation(row),axis=1)
ia_not_assets['Date_from_Built'] = ia_not_assets.apply (lambda row: date_from_built(row),axis=1)
ia_not_assets['Date_from_Renovation'] = ia_not_assets.apply (lambda row: date_from_renovation(row),axis=1)

ia_not_assets['Interval_Deal_Start_Date'] = np.NAN
ia_not_assets['Interval_Deal_End_Date'] = np.NAN

# Calculate the value for columns 'Date_to_Market' and 'Date_to_Predict'
ia_not_assets['Date_to_Market'] = ia_not_assets['Date_from_Built']
ia_not_assets['Date_to_Predict'] = ia_not_assets.apply (lambda row: date_to_predict(row),axis=1)

# Add the column "Is_Transaction_Exist", and set the value of each row to be 0 (Assets without market transaction).
ia_not_assets["Is_Transaction_Exist"] = 0

# Sort each row in AssetID increasing order.
ia_not_assets_order = ia_not_assets.sort_values(by = ['AssetID'])

# Save the data frame to result file for further checking analysis purpose.
ia_not_assets_order.to_csv('ia_not_assets_complete.csv')


In [ ]:
##### 1.6 Concat the result data frame from 1.4 and 1.5 as the main data frame for modeling, analysis and prediction.

total_asset_transactions = pd.concat([deal_asset_time_order_filter, ia_not_assets_order]).drop_duplicates()

print (total_asset_transactions.shape)
print (list(total_asset_transactions.head(0)))


In [ ]:
####### Part 2. Unsupervised learning and Data Visualization.
####### The purpose is for feature Selection and dimension reduction. 
####### Figure out the most important features.


##### 2.1 Please view the code and visualization parts of 'R' codes.
##### Please check the code and visualization result in 'Lead_Generation.pdf'.
##### Data source: 'ia_deal_assets_time_complete.csv' generated from the prior step. 

##### 2.2 Make further feature analysis, selection and dimension reduction with Information Gain.

from info_gain import info_gain
import matplotlib.pyplot as plt;

### Calculate Information Gain. Drop the fields with too little Information Gain.

# Other Optional Feature selection methods: Filtering, Wrapping, Gini-Gain.
# Feature Transformation: PCA, ICA, Randomized Projection.
# "Date to Market" can not be used to do information gain analysis
# Of all 19 fields, remove the 5 fields with the least information gain for supervised learning: IsGateway, NumberofBuildings, PortfolioCount, IsPortfolio, and Number_of_Floors. 

# Identically and independently sample 10% of total data for dimension reduction analysis to avoid bias.
sample_asset_transactions = total_asset_transactions.sample(frac=0.1, replace=False)

print (sample_asset_transactions.shape)
print ("Information gain for the below columns:")
print (list(sample_asset_transactions.head(0)))

# Set up two lists to save each field name and its Information Gain value.
field_name = []
information_gain = []

ig_YearBuilt = info_gain.info_gain( sample_asset_transactions.Is_Transaction_Exist, sample_asset_transactions.YearBuilt)
print ("YearBuilt: " + str(ig_YearBuilt))
field_name.append("YearBuilt")
information_gain.append(ig_YearBuilt)

ig_IsPortfolio = info_gain.info_gain(sample_asset_transactions.Is_Transaction_Exist, sample_asset_transactions.IsPortfolio)
print("IsPortfolio: " + str(ig_IsPortfolio))
field_name.append("IsPortfolio")
information_gain.append(ig_IsPortfolio)

ig_YearRenovated_NotNull = sample_asset_transactions [sample_asset_transactions.YearRenovated.notnull()]
ig_YearRenovated  = info_gain.info_gain(ig_YearRenovated_NotNull.Is_Transaction_Exist, ig_YearRenovated_NotNull.YearRenovated)
print ("Year_Renovated: " + str(ig_YearRenovated))
field_name.append("Year_Renovated")
information_gain.append(ig_YearRenovated)

ig_Date_from_Built = info_gain.info_gain( sample_asset_transactions.Is_Transaction_Exist, sample_asset_transactions.Date_from_Built)
print ("Date_from_Built: " + str(ig_Date_from_Built))
field_name.append("Date_from_Built")
information_gain.append(ig_Date_from_Built)

ig_Date_from_Renovation_NotNull = sample_asset_transactions [(sample_asset_transactions.Date_from_Renovation.notnull()) & (sample_asset_transactions.Date_from_Renovation != -1)]
ig_Date_from_Renovation = info_gain.info_gain(ig_Date_from_Renovation_NotNull.Is_Transaction_Exist, ig_Date_from_Renovation_NotNull.Date_from_Renovation)
print ("Date_from_Renovation: " + str(ig_Date_from_Renovation))
field_name.append("Date_from_Renovation")
information_gain.append(ig_Date_from_Renovation)

ig_NumberofFloors_NotNull = sample_asset_transactions [sample_asset_transactions.Date_from_Renovation.notnull()]
ig_NumberofFloors = info_gain.info_gain(ig_NumberofFloors_NotNull.Is_Transaction_Exist, ig_NumberofFloors_NotNull.NumberofFloors)
print("Number_of_Floors: " + str(ig_NumberofFloors))
field_name.append("Number_of_Floors")
information_gain.append(ig_NumberofFloors)

ig_NumberofBuildings_NotNull = sample_asset_transactions[sample_asset_transactions.NumberofBuildings.notnull()]
ig_NumberofBuildings = info_gain.info_gain(ig_NumberofBuildings_NotNull.Is_Transaction_Exist, ig_NumberofBuildings_NotNull.NumberofBuildings)
print("NumberofBuildings: " + str(ig_NumberofBuildings))
field_name.append("Number_of_Buildings")
information_gain.append(ig_NumberofBuildings)

ig_IsGateway_NotNull = sample_asset_transactions[sample_asset_transactions.IsGateway.notnull()]
ig_IsGateway = info_gain.info_gain(ig_IsGateway_NotNull.Is_Transaction_Exist, ig_IsGateway_NotNull.IsGateway)
print("IsGateway: " + str(ig_IsGateway))
field_name.append("Is_Gateway")
information_gain.append(ig_IsGateway)

ig_PortfolioCount_NotNull = sample_asset_transactions[sample_asset_transactions.PortfolioCount.notnull()]
ig_PortfolioCount = info_gain.info_gain(ig_PortfolioCount_NotNull.Is_Transaction_Exist, ig_PortfolioCount_NotNull.PortfolioCount)
print("PortfolioCount: " + str(ig_PortfolioCount))
field_name.append("Is_Portfolio_Count")
information_gain.append(ig_PortfolioCount)

ig_Size_Units_NotNull = sample_asset_transactions[sample_asset_transactions.Size_Units.notnull()]
ig_Size_Units = info_gain.info_gain(ig_Size_Units_NotNull.Is_Transaction_Exist, ig_Size_Units_NotNull.Size_Units)
print("Size_Units: " + str(ig_Size_Units))
field_name.append("Size_Units")
information_gain.append(ig_Size_Units)

ig_Size_Sqft_NotNull = sample_asset_transactions[sample_asset_transactions.Size_Sqft.notnull()]
ig_Size_Sqft = info_gain.info_gain(ig_Size_Sqft_NotNull .Is_Transaction_Exist, ig_Size_Sqft_NotNull.Size_Sqft)
print("Size_Sqft: " + str(ig_Size_Sqft))
field_name.append("Size_Sqft")
information_gain.append(ig_Size_Sqft)

ig_LandAreaInAcres_NotNull = sample_asset_transactions[sample_asset_transactions.LandAreaInAcres.notnull()]
ig_LandAreaInAcres = info_gain.info_gain(ig_LandAreaInAcres_NotNull.Is_Transaction_Exist, ig_LandAreaInAcres_NotNull.LandAreaInAcres)
print("LandAreaInAcres: " + str(ig_LandAreaInAcres))
field_name.append("Land_Area_InAcres")
information_gain.append(ig_LandAreaInAcres)

ig_Size_Acres_NotNull = sample_asset_transactions[sample_asset_transactions.Size_Acres.notnull()]
ig_Size_Acres = info_gain.info_gain(ig_Size_Acres_NotNull.Is_Transaction_Exist, ig_Size_Acres_NotNull.Size_Acres)
print("Size_Acres: " + str(ig_Size_Acres))
field_name.append("Size_Acres")
information_gain.append(ig_Size_Acres)

ig_Size_InLineSqft_NotNull = sample_asset_transactions[sample_asset_transactions.Size_InLineSqft.notnull()]
ig_Size_InLineSqft = info_gain.info_gain(ig_Size_InLineSqft_NotNull.Is_Transaction_Exist, ig_Size_InLineSqft_NotNull.Size_InLineSqft)
print("Size_InLineSqft: " + str(ig_Size_InLineSqft))
field_name.append("Size_InLineSqft")
information_gain.append(ig_Size_InLineSqft)

ig_AssetLatitude_NotNull = sample_asset_transactions[sample_asset_transactions.AssetLatitude.notnull()]
ig_AssetLatitude  = info_gain.info_gain(ig_AssetLatitude_NotNull.Is_Transaction_Exist, ig_AssetLatitude_NotNull.AssetLatitude)
print("AssetLatitude: "  + str(ig_AssetLatitude))
field_name.append("Asset_Latitude")
information_gain.append(ig_AssetLatitude)

ig_AssetLongitude_NotNull = sample_asset_transactions[sample_asset_transactions.AssetLongitude.notnull()]
ig_AssetLongitude = info_gain.info_gain(ig_AssetLongitude_NotNull.Is_Transaction_Exist, ig_AssetLongitude_NotNull.AssetLongitude)
print("AssetLongitude: " + str(ig_AssetLongitude))
field_name.append("Asset_Longitude")
information_gain.append(ig_AssetLongitude)

ig_Lat_city_NotNull = sample_asset_transactions[sample_asset_transactions.Lat_city.notnull()]
ig_Lat_city = info_gain.info_gain(ig_Lat_city_NotNull.Is_Transaction_Exist, ig_Lat_city_NotNull.Lat_city)
print("Lat_city: " + str(ig_Lat_city))
field_name.append("Lat_city")
information_gain.append(ig_Lat_city)

ig_Long_city_NotNull = sample_asset_transactions[sample_asset_transactions.Long_city.notnull()]
ig_Long_city = info_gain.info_gain(ig_Long_city_NotNull.Is_Transaction_Exist, ig_Long_city_NotNull.Long_city)
print("Long_city: " + str(ig_Long_city))
field_name.append("Long_city")
information_gain.append(ig_Long_city)

# Form a data frame to save the Information Gain value with its relevant field name.
ig_frames = pd.DataFrame({
    'field_name' : field_name,
    'information_gain' : information_gain })

# Sort each field name by the Information Gain decreasing order.
ig_frames_sort = ig_frames.sort_values(by =['information_gain'], ascending=False)

# Generate the figure to show each field in Information Gain decreasing order.
plt.bar(ig_frames_sort.field_name, ig_frames_sort.information_gain,  align='center', alpha=0.5)
plt.xlabel('Field Name')
plt.xticks(ig_frames_sort.field_name, rotation=90 )
plt.ylabel('Information Gain Value')
plt.title('Information Gain Value for Different Fields')
plt.savefig('information_gain.png')
plt.show()


In [ ]:
##### 2.3 Add all required columns for Statistical / Machine Learning modeling and analysis purpose. 
total_asset_transactions["Logistic_Regression"] = np.NAN
total_asset_transactions["Logistic_Regression_Accuracy"] = np.NAN
total_asset_transactions["Logistic_Regression_Accuracy_1"] = np.NAN
total_asset_transactions["Decision_Tree"] = np.NAN
total_asset_transactions["Decision_Tree_Accuracy"] = np.NAN
total_asset_transactions["Decision_Tree_Accuracy_1"] = np.NAN
total_asset_transactions["Neural_Network"] = np.NAN
total_asset_transactions["Neural_Network_Accuracy"] = np.NAN
total_asset_transactions["Neural_Network_Accuracy_1"] = np.NAN
total_asset_transactions["Support_Vector_Machine"] = np.NAN
total_asset_transactions["SVM_Accuracy"] = np.NAN
total_asset_transactions["SVM_Accuracy_1"] = np.NAN
total_asset_transactions["Boosting"] = np.NAN
total_asset_transactions["Boosting_Accuracy"] = np.NAN
total_asset_transactions["Boosting_Accuracy_1"] = np.NAN

total_asset_transactions["Bi_Normal_Distribution_Probability"] = np.NAN
total_asset_transactions["Weibull_Distribution_Probability"] = np.NAN

total_asset_transactions["Market_Probability"] = np.NAN
total_asset_transactions["Recommendation_Level"] = np.NAN


In [ ]:
####### Part 3. Build Machine Learning and Statistical models. Make analysis and predictions.

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix

##### 3.1 Identically and independently sample 10% of total data for dimension reduction analysis to avoid bias.

### 'sample_asset_transactions' is generated in Part 2, and contains 10% of random data.
###  Select the important attribuates for learning.
sample_asset_transactions_0 = sample_asset_transactions [['Date_from_Built', 'Date_from_Renovation', 'Size_Sqft', 'Size_Acres', 'Size_Units', 'LandAreaInAcres',
                                                          'PropertyTypeID','NumberofBuildings', 'AssetLatitude', 'AssetLongitude', 'Lat_city', 'Long_city',
                                                          'IsPortfolio', 'PortfolioCount', 'IsGateway', 'Date_to_Market', 'Is_Transaction_Exist']].fillna(0)

# Divides the sample data to 80% training and 20% testing randomly.
msk = np.random.rand(len(sample_asset_transactions_0)) < 0.8
SL_Train = sample_asset_transactions_0[msk]
SL_Test = sample_asset_transactions_0[~msk]

# Get the training features and labels
SL_Train_X = SL_Train.iloc[:, :-1]
SL_Train_y = SL_Train.iloc[:, -1]

# Get the testing features and labels
SL_Test_X = SL_Test.iloc[:, :-1]
SL_Test_y = SL_Test.iloc[:, -1]

### Set up multiple lists to store the result data.
model_names = []
train_accuracy_list = []
test_accuracy_list = []
test_recall_list = []
test_precision_list = []
model_std_errors = []
train_times = []
prediction_times = []

##### 3.2 Build Logistic Regression models

### Show the logistic regression theory.
### Suitable for the vector with lots of 'Null' value.
### Simple Statistical Theory (Easy to understand.)
### Higher prediction speed (Suitable for industry need.)

### To do list: select multiple different column combination to do training.
### Generate Learning Curve to show the learning process and guard against overfitting.
### Use 10-fold cross validation to improve accuracy
### Use Testing accuracy as the final accuracy  Improvement: Use the Cross Validation accuracy.
### Use 'Confusion Matrix' to evaluate accuracy to find improvement: 'Precision', 'Recall' and 'Accuracy'

# Train model
start_time = dt.datetime.now()

Logistic_model = LogisticRegression()
Logistic_model.fit(SL_Train_X, SL_Train_y)

train_times.append(dt.datetime.now() - start_time)

# Get the training and testing accuracy
predicted_train_classes = Logistic_model.predict(SL_Train_X)
predicted_test_classes = Logistic_model.predict(SL_Test_X)

train_accuracy = accuracy_score(SL_Train_y, predicted_train_classes)
test_accuracy = accuracy_score(SL_Test_y, predicted_test_classes)
test_recall = recall_score(SL_Test_y, predicted_test_classes, average='macro')
test_precision = precision_score(SL_Test_y, predicted_test_classes, average='macro')

model_names.append ("Logistic_Regression")
train_accuracy_list.append (train_accuracy)
test_accuracy_list.append (test_accuracy) 
test_recall_list.append (test_recall)
test_precision_list.append(test_precision)
model_std_errors.append(np.NAN)

parameters = Logistic_model.coef_

print (train_accuracy)
print (test_accuracy) # Most important 
print (test_recall)
print (test_precision)
print (parameters)
print (confusion_matrix(SL_Test_y, predicted_test_classes))
print (classification_report(SL_Test_y, predicted_test_classes))

# 0.9973963009516968
# 0.9967509025270758
# 0.9759358288770054
# 0.9982638888888888
# [[ 2.63335027e-02  2.19423333e-05 -6.71914521e-08 -4.25657326e-04
#   -3.87426307e-04 -2.54802891e-03 -4.03197853e-03 -1.26625430e-05
#   -8.54026471e-03  2.15459471e-02 -6.66240928e-03  1.74716598e-02
#   -4.19825292e-05 -4.53191319e-05 -4.08402082e-05 -2.64754610e-02]]
# [[2583    0]
#  [   9  178]]
#               precision    recall  f1-score   support

#            0       1.00      1.00      1.00      2583
#            1       1.00      0.95      0.98       187

#    micro avg       1.00      1.00      1.00      2770
#    macro avg       1.00      0.98      0.99      2770
# weighted avg       1.00      1.00      1.00      2770

# Make the predictions
asset_transactions_full = total_asset_transactions [['Date_from_Built', 'Date_from_Renovation', 'Size_Sqft', 'Size_Acres', 'Size_Units', 'LandAreaInAcres',
                                                          'PropertyTypeID', 'NumberofBuildings', 'AssetLatitude', 'AssetLongitude', 'Lat_city', 'Long_city',
                                                          'IsPortfolio', 'PortfolioCount', 'IsGateway', 'Date_to_Predict']].fillna(0)

start_time = dt.datetime.now()
total_asset_transactions["Logistic_Regression"] = Logistic_model.predict(asset_transactions_full)
prediction_times.append(dt.datetime.now() - start_time)

# Save the prediction result.
total_asset_transactions["Logistic_Regression_Accuracy"] = test_accuracy
total_asset_transactions["Logistic_Regression_Accuracy_1"] = test_accuracy
total_asset_transactions["Logistic_Regression_Accuracy_1"] = np.where(total_asset_transactions["Logistic_Regression"] == 1, total_asset_transactions["Logistic_Regression_Accuracy"], 1 - total_asset_transactions["Logistic_Regression_Accuracy"])

### Coefficient is reasonable, since there is not much large, prevent overfitting,
### Also, Date_to_Market has almost the largest coefficient, indicating that it is most important.
### Model Issue: The more transaction a property has, the shorter it will be into the market next time.
### The longer time it does not have transaciton, it will not be easiliy into the market.  (May not totally agree to our basic instinct)


In [ ]:
##### 3.3 Build Decision Tree models

from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

# Do not use too much fields, make the tree not to be too complex, otherwise, it may overfitting.
# Decision tree pruning is necessary sometimes to reduce ovefitting.
# Tune the value of pruning, max no. of instance per nodes (small value can reduce complexity), confidence factor (small value means more pruning), subtree lifting, etc.

# Train model
start_time = dt.datetime.now()

Decision_Tree = DecisionTreeClassifier()
Decision_Tree.fit(SL_Train_X, SL_Train_y)

train_times.append(dt.datetime.now() - start_time)

# Get the training and testing accuracy
predicted_train_classes = Decision_Tree.predict(SL_Train_X)
predicted_test_classes = Decision_Tree.predict(SL_Test_X)
train_accuracy = accuracy_score(SL_Train_y, predicted_train_classes)
test_accuracy = accuracy_score(SL_Test_y, predicted_test_classes)
test_recall = recall_score(SL_Test_y, predicted_test_classes, average='macro')
test_precision = precision_score(SL_Test_y, predicted_test_classes, average='macro')

model_names.append ("Decision_Tree")
train_accuracy_list.append (train_accuracy)
test_accuracy_list.append (test_accuracy) 
test_recall_list.append (test_recall)
test_precision_list.append(test_precision)
model_std_errors.append(np.NAN)

tree.export_graphviz(Decision_Tree, out_file='decision_tree.dot')

print (train_accuracy)
print (test_accuracy)
print (test_recall)
print (test_precision)
print (confusion_matrix(SL_Test_y, predicted_test_classes))
print (classification_report(SL_Test_y, predicted_test_classes))

# 1.0   (Overfitting happens!)
# 0.9379061371841155
# 0.760846836887009
# 0.7533260549178564
# [[2494   89]
#  [  83  104]]
#               precision    recall  f1-score   support

#            0       0.97      0.97      0.97      2583
#            1       0.54      0.56      0.55       187

#    micro avg       0.94      0.94      0.94      2770
#    macro avg       0.75      0.76      0.76      2770
# weighted avg       0.94      0.94      0.94      2770

# Make the predictions
start_time = dt.datetime.now()
total_asset_transactions["Decision_Tree"] = Decision_Tree.predict(asset_transactions_full)
prediction_times.append(dt.datetime.now() - start_time)

# Save the prediction result.
total_asset_transactions["Decision_Tree_Accuracy"] = test_accuracy
total_asset_transactions["Decision_Tree_Accuracy_1"] = test_accuracy
total_asset_transactions["Decision_Tree_Accuracy_1"] = np.where(total_asset_transactions["Decision_Tree"] == 1, total_asset_transactions["Decision_Tree_Accuracy"], 1 - total_asset_transactions["Decision_Tree_Accuracy"])


In [ ]:
##### 3.4 Build Neural Network (Multiple Layer Perceptron) models

from sklearn.neural_network import MLPClassifier  # Multiple Layer Perceptron

### Default layer to be 1, the hidden nodes are using the formula (dimensions + class)/2 = 8
### Do not use too much fields, make the network not to be too complex, otherwise, it may overfit.
### Tune the value of hidden layer, no. of nodes each layer, max training iterations, etc.

# Issue: The more transaction a property has, the shorter it will be into the market next time.
# The longer time it does not have transaciton, it will not be easiliy into the market.  (May not totally agree to our basic instinct)

# Train model
start_time = dt.datetime.now()
Neural_Network_Classifier = MLPClassifier(hidden_layer_sizes=(8, ), max_iter=1000, momentum=0.3)

# Default: MLPClassifier(hidden_layer_sizes=(100, ), activation=’relu’, solver=’adam’, alpha=0.0001, batch_size=’auto’, learning_rate=’constant’, learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10)[source]¶
Neural_Network_Classifier.fit(SL_Train_X, SL_Train_y)
train_times.append(dt.datetime.now() - start_time)

# Get the training and testing accuracy
predicted_train_classes = Neural_Network_Classifier.predict(SL_Train_X)
predicted_test_classes = Neural_Network_Classifier.predict(SL_Test_X)
train_accuracy = accuracy_score(SL_Train_y, predicted_train_classes)
test_accuracy = accuracy_score(SL_Test_y, predicted_test_classes)
test_recall = recall_score(SL_Test_y, predicted_test_classes, average='macro')
test_precision = precision_score(SL_Test_y, predicted_test_classes, average='macro')

model_names.append ("Neural_Network")
train_accuracy_list.append (train_accuracy)
test_accuracy_list.append (test_accuracy) 
test_recall_list.append (test_recall)
test_precision_list.append(test_precision)
model_std_errors.append(np.NAN)

print (train_accuracy)
print (test_accuracy)
print (test_recall)
print (test_precision)
print (confusion_matrix(SL_Test_y, predicted_test_classes))
print (classification_report(SL_Test_y, predicted_test_classes))

# 0.9829412820973245
# 0.9844765342960289
# 0.8949476316764695
# 0.9793936713441358
# [[2579    4]
#  [  39  148]]
#               precision    recall  f1-score   support

#            0       0.99      1.00      0.99      2583
#            1       0.97      0.79      0.87       187

#    micro avg       0.98      0.98      0.98      2770
#    macro avg       0.98      0.89      0.93      2770
# weighted avg       0.98      0.98      0.98      2770

# Make the predictions
start_time = dt.datetime.now()
total_asset_transactions["Neural_Network"] = Neural_Network_Classifier.predict(asset_transactions_full)
prediction_times.append(dt.datetime.now() - start_time)

# Save the prediciton result
total_asset_transactions["Neural_Network_Accuracy"] = test_accuracy
total_asset_transactions["Neural_Network_Accuracy_1"] = test_accuracy
total_asset_transactions["Neural_Network_Accuracy_1"] = np.where(total_asset_transactions["Neural_Network"] == 1, total_asset_transactions["Decision_Tree_Accuracy"], 1 - total_asset_transactions["Neural_Network_Accuracy"])


In [ ]:
##### 3.5 Build Support Vector Machine models

### To do: Change Kernal methods: RBF (Radial Basis Function); Polynomial
### Linear kernal function performs comparably to Logistic Regression in practice. However, SVM may perform better due to its maximum margin and optimal hyperplane,
### SVM is less sensitive to outliers.
### SVM is resilient to overfitting.

from sklearn.svm import SVC

# Train model
start_time = dt.datetime.now()
SVM_classifier = SVC(kernel='linear')
SVM_classifier.fit(SL_Train_X, SL_Train_y)
train_times.append(dt.datetime.now() - start_time)

# Get the training and testing accuracy
predicted_train_classes = SVM_classifier.predict(SL_Train_X)
predicted_test_classes = SVM_classifier.predict(SL_Test_X)
train_accuracy = accuracy_score(SL_Train_y, predicted_train_classes)
test_accuracy = accuracy_score(SL_Test_y, predicted_test_classes)
test_recall = recall_score(SL_Test_y, predicted_test_classes, average='macro')
test_precision = precision_score(SL_Test_y, predicted_test_classes, average='macro')

model_names.append ("Support_Vector_Machine")
train_accuracy_list.append (train_accuracy)
test_accuracy_list.append (test_accuracy) 
test_recall_list.append (test_recall)
test_precision_list.append(test_precision)
model_std_errors.append(np.NAN)

print (train_accuracy)
print (test_accuracy)
print (test_recall)
print (test_precision)
print (confusion_matrix(SL_Test_y, predicted_test_classes))
print (classification_report(SL_Test_y, predicted_test_classes))

# 1.0
# 1.0
# 1.0
# 1.0
# [[2583    0]
#  [   0  187]]
#               precision    recall  f1-score   support

#            0       1.00      1.00      1.00      2583
#            1       1.00      1.00      1.00       187

#    micro avg       1.00      1.00      1.00      2770
#    macro avg       1.00      1.00      1.00      2770
# weighted avg       1.00      1.00      1.00      2770


# Make the predictions
start_time = dt.datetime.now()
total_asset_transactions["Support_Vector_Machine"] = SVM_classifier.predict(asset_transactions_full)
prediction_times.append(dt.datetime.now() - start_time)

# Save the prediciton result
total_asset_transactions["SVM_Accuracy"] = test_accuracy
total_asset_transactions["SVM_Accuracy_1"] = test_accuracy
total_asset_transactions["SVM_Accuracy_1"] = np.where(total_asset_transactions["Support_Vector_Machine"] == 1, total_asset_transactions["SVM_Accuracy"], 1 - total_asset_transactions["SVM_Accuracy"])

In [ ]:
##### 3.6 Build Boosting model - Ada Boost

### Boosting: Multiple weak classifers work together to determine the classifier output. Use Ada Boost.
### Higher accuracy due to expanded decision hyperplan.
### It could normally take much longer time to train.
### The next weak classifier is based on the trained weak classifier, and mistrained instance will have higher weight.
### To do: Change weak classifiers, and the parameter of each weak classifier.

from sklearn.ensemble import AdaBoostClassifier

# Train model
seed = 10
num_trees = 20
start_time = dt.datetime.now()
Boosting_Model = AdaBoostClassifier(n_estimators=num_trees, random_state=seed)
Boosting_Model.fit(SL_Train_X, SL_Train_y)
train_times.append(dt.datetime.now() - start_time)

# Get the training and testing accuracy
predicted_train_classes = Boosting_Model.predict(SL_Train_X)
predicted_test_classes = Boosting_Model.predict(SL_Test_X)
train_accuracy = accuracy_score(SL_Train_y, predicted_train_classes)
test_accuracy = accuracy_score(SL_Test_y, predicted_test_classes)
test_recall = recall_score(SL_Test_y, predicted_test_classes, average='macro')
test_precision = precision_score(SL_Test_y, predicted_test_classes, average='macro')

model_names.append ("Boosting")
train_accuracy_list.append (train_accuracy)
test_accuracy_list.append (test_accuracy) 
test_recall_list.append (test_recall)
test_precision_list.append(test_precision)
model_std_errors.append(np.NAN)

print (train_accuracy)
print (test_accuracy)
print (test_recall)
print (test_precision)
print(confusion_matrix(SL_Test_y, predicted_test_classes))
print(classification_report(SL_Test_y, predicted_test_classes))

# 0.9478362363081343
# 0.9447653429602888
# 0.600829984617646
# 0.9250715682167295
# [[2579    4]
#  [ 149   38]]
#               precision    recall  f1-score   support

#            0       0.95      1.00      0.97      2583
#            1       0.90      0.20      0.33       187

#    micro avg       0.94      0.94      0.94      2770
#    macro avg       0.93      0.60      0.65      2770
# weighted avg       0.94      0.94      0.93      2770

# Make the predictions
start_time = dt.datetime.now()
total_asset_transactions["Boosting"] = Boosting_Model.predict(asset_transactions_full)
prediction_times.append(dt.datetime.now() - start_time)

# Save the prediciton result
total_asset_transactions["Boosting_Accuracy"] = test_accuracy
total_asset_transactions["Boosting_Accuracy_1"] = test_accuracy
total_asset_transactions["Boosting_Accuracy_1"] = np.where(total_asset_transactions["Boosting"] == 1, total_asset_transactions["Boosting_Accuracy"], 1 - total_asset_transactions["Boosting_Accuracy"])

#####################################################################
### Several classifiers are not selected as below:
### KNN is not selected due to the difficulty of gaining distance for each dimension, how single dimension distance plays the role in the final distance. Also, avoid curse of dimensionality.
### HMM: Unknown Transition, Imition probability.
### Naive Bayes: Condition Independent.


In [ ]:
####### 3.7 Build Statistical Modeling - Linear Regression

### Based on the data analysis and visualization, 'Date_from_Built' and 'Date_to_Market' has strong linear relationship.
### Calculate the Standard error of this linear regression
### Change the area (region, state, city, etc) for the modeling of specific areas.
### Add more measurement factors in the linear regression model.

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

# Filter the transaction record based on 'Date_to_Market' values.
deal_asset_time_order_filter_new = deal_asset_time_order_filter[deal_asset_time_order_filter.Date_to_Market >= 90]

# Initiate Linear regression model
start_time = dt.datetime.now()
Linear_Regression = linear_model.LinearRegression()

# Train the model using the training sets
Linear_Regression.fit(deal_asset_time_order_filter_new[['Date_from_Built']], deal_asset_time_order_filter_new['Date_to_Market'])
train_times.append(dt.datetime.now() - start_time)

print (Linear_Regression.coef_)
print (Linear_Regression.intercept_)

# [0.9095816]
# -1391.8770039558985

# Predict 'Date_to_Market' based on the linear regression model (for training dataset)
Date_Linear_Predict = Linear_Regression.predict(deal_asset_time_order_filter_new[['Date_from_Built']])

# Get the standard deviation of the linear regression model (for training dataset)
std_error = np.sqrt(mean_squared_error(Date_Linear_Predict, deal_asset_time_order_filter_new['Date_to_Market']))
r2_coefficient = r2_score(Date_Linear_Predict, deal_asset_time_order_filter_new['Date_to_Market'])

print ( "Prediction Standard Error: " + str(std_error))
print ("R2 Coefficient of Determination: "+ str(r2_coefficient))

# Prediction Standard Error: 3438.454228361336
# R2 Coefficient of Determination: 0.7739593610255957

plt.scatter(deal_asset_time_order_filter_new['Date_from_Built'], deal_asset_time_order_filter_new['Date_to_Market'])
plt.plot(deal_asset_time_order_filter_new['Date_from_Built'], Date_Linear_Predict)
plt.xlabel('Date to Built')
plt.ylabel('Date to Market')
plt.title('Linear Regression for Date to Market vs Date to Built')
plt.savefig('linear_regression.png')
plt.show()

model_names.append ("Linear_Regression")
model_std_errors.append(std_error)
train_accuracy_list.append (np.NAN)
test_accuracy_list.append (np.NAN) 
test_recall_list.append (np.NAN)
test_precision_list.append(np.NAN)
    
start_time = dt.datetime.now()

# Make 'Date_to_Market' predictions for the whole data set
total_Date_Linear_Predict = Linear_Regression.predict(total_asset_transactions[['Date_from_Built']])
prediction_times.append(dt.datetime.now() - start_time)

import scipy.stats as st

# Predict and save the market probability based on the 'Date_to_Predict' column
total_asset_transactions["Linear_Regression_Probability"] = st.norm.cdf((total_asset_transactions['Date_to_Predict'] - total_Date_Linear_Predict) / std_error)

In [ ]:
####### 3.8 Build Statistical Modeling - Bi-modal Normal distribution.

##### Use the combination of 2 normal distribution, based on the Bi-modal normal distribution of 'Date_to_Market' curve.

### Calculate the standard error of 'Date_to_Market'
start_time = dt.datetime.now()
norm_std_error = np.std(deal_asset_time_order_filter_new['Date_to_Market'])
train_times.append(dt.datetime.now() - start_time)

model_names.append ("Bi-modal_Normal_Distribution")
model_std_errors.append(norm_std_error)
train_accuracy_list.append (np.NAN)
test_accuracy_list.append (np.NAN) 
test_recall_list.append (np.NAN)
test_precision_list.append(np.NAN)

# print (norm_std_error)
# 8007

### Predict and save the market probability based on the 'Date_to_Predict' column 
start_time = dt.datetime.now()
total_asset_transactions["Bi_Normal_Distribution_Probability"]= 0.5 * (st.norm(1500, norm_std_error).cdf(total_asset_transactions['Date_to_Predict'] ) + st.norm(11000, norm_std_error).cdf(total_asset_transactions['Date_to_Predict'] ))
prediction_times.append(dt.datetime.now() - start_time)

from sklearn import mixture

clf = mixture.GaussianMixture(n_components=2, covariance_type='full')
clf.fit(deal_asset_time_order_filter_new[['Date_to_Market']])
m1, m2 = clf.means_
w1, w2 = clf.weights_
c1, c2 = clf.covariances_ 

print (m1)
print (m2)
print (w1)
print (w2)
print (c1)
print (c2)

# [20530.77761653]
# [7124.37071675]
# 0.14143008521606848
# 0.8585699147839324
# [[1.41457215e+08]]
# [[25969782.696412]]


In [ ]:
####### 3.9 Build Statistical Modeling - Weibull distribution.
### Weibull distribution to model 'Date_to_Market'.
### Use the linear combination of 'Date_from_Built', 'Date_from_Renovation' and 'PortfolioCount' as the beta (shape) value.

### Calculate the beta (shape), location, and eta (scale) value.
start_time = dt.datetime.now()
beta, location, eta = st.weibull_min.fit(deal_asset_time_order_filter_new['Date_to_Market'].astype(np.float64))
train_times.append(dt.datetime.now() - start_time)
print (beta)
print (location)
print (eta)

std_error = np.sqrt(st.weibull_min.std(c= beta, loc=location, scale=eta))
model_names.append ("Weibull_Distribution")
model_std_errors.append(std_error)
train_accuracy_list.append (np.NAN)
test_accuracy_list.append (np.NAN) 
test_recall_list.append (np.NAN)
test_precision_list.append(np.NAN)

print (std_error)

# 0.10451769749916562
# 89.99999999999997
# 2.5503610789148983
# 32986.09534077522

### Set up Linear Regression to calculate the 3 coefficients. 
linear_fit = linear_model.LinearRegression()          
linear_fit.fit(deal_asset_time_order_filter_new[['Date_from_Built', 'Date_from_Renovation', 'PortfolioCount']].fillna(0), eta * np.ones(deal_asset_time_order_filter_new.shape[0]))

### Predict and save the 'Date_to_Market' probability
eta_factor_records = total_asset_transactions[['Date_from_Built', 'Date_from_Renovation', 'PortfolioCount']].fillna(0)

start_time = dt.datetime.now()
total_asset_transactions['eta_predict'] = linear_fit.predict(eta_factor_records[['Date_from_Built', 'Date_from_Renovation', 'PortfolioCount']])
prediction_times.append(dt.datetime.now() - start_time)

total_asset_transactions["Weibull_Distribution_Probability"] = st.weibull_min(c=beta, loc=location, scale=total_asset_transactions[['eta_predict']]).cdf(total_asset_transactions[["Date_to_Predict"]])


In [ ]:
### Generate the image to compare the accuracy, precision, recall of the above 5 Machine Learning algorithms.
### Testing accuracy is most important.

# Form a data frame to save the Information Gain value with its relevant field name.
model_performance_frames = pd.DataFrame({
    'model_name' : model_names,
    'train_accuracy' : train_accuracy_list,
    'test_accuracy' : test_accuracy_list,
    'test_recall' : test_recall_list,
    'test_precision' : test_precision_list,
    'standard_error' : model_std_errors})

# Sort each row of Machine Learning models by the Testing Accuracy decreasing order.
ml_model_performance_sort = model_performance_frames.loc[0:4].sort_values(by =['test_accuracy'], ascending=False)
print (ml_model_performance_sort)

ml_model_performance_sort.plot(x = "model_name", y=["test_accuracy", "train_accuracy", "test_recall", "test_precision"], kind="bar")
plt.savefig('ml_model_performance.png')
plt.show()


In [ ]:
# Sort each row of Statistical models by the Testing Accuracy decreasing order.
statistic_model_performance_sort = model_performance_frames.loc[5:8].sort_values(by =['standard_error'], ascending=True)
print (statistic_model_performance_sort)

# Generate the figure to show the performance to each statistical model by the standard error increasing order.
plt.bar(statistic_model_performance_sort.model_name, statistic_model_performance_sort.standard_error,  align='center', alpha=0.5)
plt.xlabel('Model Name')
plt.xticks(statistic_model_performance_sort.model_name, rotation=30 )
plt.ylabel('Standard Error')
plt.title('Statistical Model Performance Comparison')
plt.savefig('statistic_model_performance.png')
plt.show()

In [ ]:
# Draw the figure to show the training and prediction time performance for each model

# Form a data frame to save the train and prediction time performance with its relevant model name.

time_performance_frames = pd.DataFrame({
    'model_name' : model_names,
    'train_time' : train_times,
    'prediction_time' : prediction_times})

# Sort each row of model by prediciton time ascending order.
time_performance_frames_sort_0 = time_performance_frames.sort_values(by = ['prediction_time'], ascending=True)
print (time_performance_frames_sort_0)

time_performance_frames_sort_0.plot(x = "model_name", y=["prediction_time"], kind="bar")
plt.savefig('model_prediction_time_performance.png')
plt.show()

# Sort each row of model by training time ascending order.
time_performance_frames_sort_1 = time_performance_frames.sort_values(by = ['train_time'], ascending=True)
print (time_performance_frames_sort_1)

time_performance_frames_sort_1.plot(x = "model_name", y=["train_time"], kind="line")
plt.xticks (rotation = 90)

plt.savefig('model_train_time_performance.png')
plt.show()


In [ ]:
##### 3.11 Final processing of all model and prediction results. 

# Make recommendation based on market probability
# 0.00 - 0.25: Highly not recommend; 0.25 -0.5: Not recommend; 0.5 - 0.75: Recommend; >=0.75 Highly recommend

print  (list(total_asset_transactions.head(0)))

def market_recommendation (row):
    if row['Market_Probability'] >= 0.0000 and row['Market_Probability'] < 0.25:
        return "Highly Not Recommend"
    elif row['Market_Probability'] < 0.50:
        return "Not Recommend"
    elif row['Market_Probability'] < 0.75:
        return "Recommend"
    elif row['Market_Probability'] >= 0.75:
        return "Highly Recommend"
    else:
        return np.NAN

# Calculate and save the 'Market_Probability' based on the average probability value from the above 8 methods. 
total_asset_transactions['Market_Probability'] = 0.125 * (total_asset_transactions["Logistic_Regression_Accuracy_1"] + total_asset_transactions["Decision_Tree_Accuracy_1"] + total_asset_transactions["Neural_Network_Accuracy_1"] + total_asset_transactions["SVM_Accuracy_1"] + total_asset_transactions["Boosting_Accuracy_1"]
                                                 + total_asset_transactions["Linear_Regression_Probability"] + total_asset_transactions["Bi_Normal_Distribution_Probability"] + total_asset_transactions["Weibull_Distribution_Probability"])
# Calculate and save the 'Recommendation_Level' value based on the 'Market_Probability' value.
total_asset_transactions['Recommendation_Level'] = total_asset_transactions.apply (lambda row: market_recommendation(row),axis=1)

total_asset_transactions = total_asset_transactions [['AssetID', 'ClientCount', 'City', 'State', 'CountryID', 'CityID', 'State.1', 'Country', 'Zip', 'PropertyTypeID', 'PropertyType', 'PropertyTypeGroup', 'YearBuilt', 'AssetLatitude', 'AssetLongitude', 'LandAreaInAcres', 'NumberofFloors', 'NumberofBuildings', 'YearRenovated', 'Size_Acres', 'Size_InLineSqft', 'IsPortfolio', 'PortfolioCount', 'Lat_city', 'Long_city', 'IsGateway', 'Region', 'Size_Sqft', 'Size_Units', 'PropertySize', 'UnitType', 'ClassDateStamp', 'Market_name', 'CityID.1', 'DealID', 'AccountingDate', 'BuyerSelectionDate', 'InitialBidDueDate', 'MarketingDate', 'HireDate', 'Deal_Start_Date', 'Deal_End_Date', 'Current_Date', 'Prediction_Date', 'Year_Built', 'Year_Renovated', 'Date_Built', 'Date_Renovated', 'Date_from_Built', 'Date_from_Renovation', 'Date_to_Market', 'Date_to_Predict', 'Is_Transaction_Exist', 'Logistic_Regression', 'Logistic_Regression_Accuracy', 'Logistic_Regression_Accuracy_1', 'Decision_Tree', 'Decision_Tree_Accuracy', 'Decision_Tree_Accuracy_1', 'Neural_Network', 'Neural_Network_Accuracy', 'Neural_Network_Accuracy_1', 'Support_Vector_Machine', 'SVM_Accuracy', 'SVM_Accuracy_1', 'Boosting', 'Boosting_Accuracy', 'Boosting_Accuracy_1', 'Linear_Regression_Probability', 'Bi_Normal_Distribution_Probability', 'Weibull_Distribution_Probability', 'Market_Probability', 'Recommendation_Level']]
total_asset_transactions.to_csv ("Asset_Deal_Prediction_Full_Result.csv")

asset_market__probability_prediction = total_asset_transactions [['AssetID', 'Market_Probability', 'Recommendation_Level']]

asset_market__probability_prediction = asset_market__probability_prediction[asset_market__probability_prediction.Market_Probability.notnull() & asset_market__probability_prediction.Recommendation_Level.notnull()].drop_duplicates()
asset_market__probability_prediction.to_csv ("Asset_Market_Prediction.csv")

print ("Prediction about IA Assets 12-months to market finishes.")


### To do list: Other alterative analysis from the results of the 8 modeling methods. Examples: Voting, Weighted Combination. 
### Calculate the error of each model, find the best fit model, and use its result as the final prediciton result.
### Use different combination (linear & non-linear) in parameter modeling and estimation.

In [ ]:
### Visualize the final recommendation result.

asset_market__probability_prediction = pd.read_csv("Asset_Market_Prediction.csv")

recommendation = ['Highly Recommend', 'Recommend', 'Not Recommend', 'Highly Not Recommend']
count = []
count.append(asset_market__probability_prediction[asset_market__probability_prediction.Recommendation_Level == 'Highly Recommend'].shape[0])
count.append(asset_market__probability_prediction[asset_market__probability_prediction.Recommendation_Level == 'Recommend'].shape[0])
count.append(asset_market__probability_prediction[asset_market__probability_prediction.Recommendation_Level == 'Not Recommend'].shape[0])
count.append(asset_market__probability_prediction[asset_market__probability_prediction.Recommendation_Level == 'Highly Not Recommend'].shape[0])

Recommendtion_counts = pd.DataFrame({
    'Recommendation_Level' : recommendation,
    'Count' : count })

Recommendtion_counts.plot(x = "Recommendation_Level", y="Count", kind="bar")
plt.savefig('final_recommendation.png')
plt.show()